In [1]:
#Import libraries needed
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import numpy as np
import cv2
import os

from keras.backend import categorical_crossentropy
from tensorflow.keras.preprocessing import image


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import *
from keras.models import Sequential


dataset_train = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)

train_path = 'C:/Users/asus/Downloads/DATASET COVID 19/train'
train_dataset = dataset_train.flow_from_directory(train_path, 
                                          target_size = (224, 224), 
                                          batch_size = 32, 
                                          class_mode = 'categorical')

Found 249 images belonging to 3 classes.


In [3]:
dataset_test = ImageDataGenerator(rescale=1./255)
test_path = 'C:/Users/asus/Downloads/DATASET COVID 19/test'
test_dataset = dataset_train.flow_from_directory(test_path, 
                                         target_size = (224, 224), 
                                         batch_size = 32, 
                                         class_mode = 'categorical')

Found 65 images belonging to 3 classes.


In [4]:
#modelling of data
model = Sequential()
model.add(Conv2D(32,
                 kernel_size=(3,3),
                 activation='relu',
                 input_shape=(224,224,3)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 110, 110, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                    

In [5]:
#running of epoch
history = model.fit_generator(train_dataset,steps_per_epoch=7,epochs = 5,validation_data = test_dataset,validation_steps=2)

C:\Users\lborb\AppData\Local\Temp\ipykernel_12900\2625774409.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_dataset,steps_per_epoch=7,epochs = 5,validation_data = test_dataset,validation_steps=2)


Epoch 1/5
7/7 [==============================] - 106s 16s/step - loss: 1.7723 - accuracy: 0.3871 - val_loss: 1.0954 - val_accuracy: 0.2969
Epoch 2/5
7/7 [==============================] - 81s 11s/step - loss: 1.0299 - accuracy: 0.5115 - val_loss: 1.0003 - val_accuracy: 0.5781
Epoch 3/5
7/7 [==============================] - 84s 12s/step - loss: 0.7796 - accuracy: 0.6518 - val_loss: 0.8566 - val_accuracy: 0.5781
Epoch 4/5
7/7 [==============================] - 98s 14s/step - loss: 0.6911 - accuracy: 0.7005 - val_loss: 0.7338 - val_accuracy: 0.6250
Epoch 5/5
7/7 [==============================] - 89s 13s/step - loss: 0.6080 - accuracy: 0.7465 - val_loss: 0.6619 - val_accuracy: 0.6719


In [6]:
#determining the accuracy
accuracy_rate = model.evaluate(train_dataset)
print('Test accuracy:', accuracy_rate[1])

8/8 [==============================] - 32s 4s/step - loss: 0.4856 - accuracy: 0.8313
Test accuracy: 0.8313252925872803


In [7]:
path_dir = 'C:/Users/asus/Downloads/DATASET COVID 19'

def loadImages(path, folder):
    data_files = sorted([os.path.join(path, folder, file)
                        for file in os.listdir(path + folder + '/')
                        if (file.endswith('.jpg') or file.endswith('.jpeg'))])
    return data_files

v_data = loadImages(path_dir, 'validation')

In [9]:
g = v_data
f = []
images = []

for x in g:
    img = cv2.imread(x)
    img = img[:, :, :3]
    images.append(img)

    width = 224
    height = 224
    dim = (width, height)

    resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
    f.append(resized)

f = np.array(f)
f = f/255

In [11]:
prediction = model.predict(f)
predictions = np.argmax(prediction, axis=1)
print(predictions)


1/1 [==============================] - 1s 605ms/step
[1 1 0 2 2 2 0 0 2 0]
